In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../data/ex.csv', sep=',')

In [2]:
df

,Unnamed: 0,AC,coh,dis,sex,imm,mrt,p,pp,ex60
0,0,60,1926,0,99,99,99,99,99,23.361782
1,1,60,1926,1,99,99,99,99,99,18.435131
2,2,60,1930,0,99,99,99,99,99,24.014652
3,3,60,1930,1,99,99,99,99,99,19.027926
4,4,60,1935,0,99,99,99,99,99,24.685810
...,...,...,...,...,...,...,...,...,...,...
31315,31315,65,1950,1,1,1,2,5,1,21.379046
31316,31316,65,1950,1,1,1,2,5,2,21.645223
31317,31317,65,1950,1,1,1,2,5,3,21.806265
31318,31318,65,1950,1,1,1,2,5,4,22.004739


    Retrieve exepected lifespan for given characteristics, both at AC=60 and AC=65

In [103]:
def get_exp_ls(coh,sex,dis,imm,mrt,p,pp):
    life_exp=[]
    for num in [60, 65]:
        filter = (df[
        (df['AC'] == num) &          # 60 61 62 63 64 65
        (df['coh'] == coh) &      # 1926 1930 1935 1940 1945 1950
        (df['sex'] == sex) &         # 0(w) 1(m) No
        (df['dis'] == dis) &         # 0(no) 1(yes)
        (df['imm'] == imm) &         # 0(no) 1(yes) No
        (df['mrt'] == mrt) &         # 0(div/single) 1(marr) 2(widow) No
        (df['p'] == p) &           # 1 2 3 4 5 No
        (df['pp'] == pp)])          # 0(no) 1 2 3 4 5 No
        if num==60:
            sum_60 = filter['ex60'].describe().loc['mean']
            life_exp.append(sum_60)
        elif num==65:
            sum_65 = filter['ex60'].describe().loc['mean']
            life_exp.append(sum_65)
    return life_exp

In [166]:
test=get_exp_ls(1926,1,1,0,2,1,0)
print(test)

[13.663432735556556, 14.335483471812422]


    Calculate the present value of a lifetime annuity with monthly payments

In [99]:
def present_value_lifetime_annuity(payment_per_period, annual_interest_rate, life_expectancy_years):
    
    monthly_interest_rate = annual_interest_rate / 12  # Convert annual interest rate to monthly
    total_periods = life_expectancy_years * 12  # Convert life expectancy to months

    if monthly_interest_rate == 0:
        present_value = payment_per_period * total_periods
    else:
        present_value = payment_per_period / monthly_interest_rate * (1 - (1 / (1 + monthly_interest_rate) ** total_periods))
    return present_value

    Calculate the cumulative value of a given cash flow over a range of ages with monthly compounding interest

In [100]:
def cumulative_value_cashflow(cash_flow_per_month, annual_interest_rate, start_age, end_age):

    monthly_interest_rate = annual_interest_rate / 12
    num_months = int((end_age - start_age) * 12)  # Convert age range to months and exclude the end month
    cumulative_value = 0
    for month in range(num_months):
        present_value = cash_flow_per_month * ((1 + monthly_interest_rate) ** month)
        cumulative_value += present_value
    return cumulative_value

    Calculate the reduced benefit induced by early claiming

In [101]:
def calculate_reduced_benefit(original_benefit, retirement_age, reduction_rate):
    if retirement_age >= 65:
        return original_benefit  # No reduction if retirement age is 65 or older
    else:
        reduction_percent = (65 - retirement_age) * reduction_rate * 12  # Annual reduction rate
        reduced_benefit = original_benefit * (1 - reduction_percent / 100)
        return reduced_benefit

    Compare the values at ages 60 and 65 for different life expectancies

In [102]:
def compare_values(rrq_65, annual_interest_rate, start_age, end_age, adj_factor,coh,sex,dis,imm,mrt,p,pp):

    comparisons = {}
    life_expectancies=get_exp_ls(coh,sex,dis,imm,mrt,p,pp)
    for life_expectancy in life_expectancies:
        # Calculate the reduced RRQ benefit at age 60
        rrq_60 = calculate_reduced_benefit(rrq_65, start_age, adj_factor)

        # Calculate the cumulative value at age 60
        cumulative_value_60 = cumulative_value_cashflow(rrq_60, annual_interest_rate, start_age, end_age) + present_value_lifetime_annuity(rrq_60, annual_interest_rate, life_expectancy)

        # Calculate the present value of a lifetime annuity at age 65
        val_claim_65 = present_value_lifetime_annuity(rrq_65, annual_interest_rate, life_expectancy)

        comparisons[life_expectancy] = {
            "Value at 60": cumulative_value_60,
            "Value at 65": val_claim_65
        }

    return comparisons

    Fill in characteristics of pension and of person, returns value of annuity

In [165]:
rrq_65 = 1000
annual_interest_rate = 0.03
start_age = 60
end_age = 65
adj_factor = .6
coh = 1926       #1926 1930 1935 1940 1945 1950
sex = 1          #0=female, 1=male, 99=model estimated without the characteristic
dis = 1          #0=has not claimed disability
imm = 0          #0=not immigrant, 99=model estimated without the characteristic
mrt = 2          #1=divorced/single, 1=married, 2=widowed, 99=model estimated without the characteristic
p = 1            #public pension in quintiles (1 to 5), 99=model estimated without the characteristic
pp = 0           #private pension in quintiles (0 to 5), 0=no private pension, 99=model estimated without the characteristic


comparison_results = compare_values(rrq_65, annual_interest_rate, start_age, end_age, adj_factor, coh, sex, dis, imm, mrt, p ,pp)

life_expectancies = list(comparison_results.keys())
values_at_60 = [comparison_results[life_exp]["Value at 60"] for life_exp in life_expectancies]
print("Value of annuity at 60yo when w/ exp_ls@60",round(values_at_60[0],1))
print("Value of annuity when claimed: @60 w/ exp_ls@60",round(values_at_65[0],1))

Value of annuity when claimed: @60 w/ exp_ls@60 127376.0
Value of annuity when claimed: @65 w/ exp_ls@60 134378.3
